In [1]:
import pandas as pd
import sqlite3
import warnings 
from IPython.display import display
warnings.simplefilter('ignore')

### Look at our dataframes

In [2]:
df_metadata = pd.read_csv('../data/metadata.csv')
df_genstudio = pd.read_csv('../data/genstudio.csv')
print('metadata shape:', df_metadata.shape)
print('genstudio shape:', df_genstudio.shape)
display(df_metadata.head(), df_genstudio.head())

metadata shape: (841, 4)
genstudio shape: (2000000, 20)


,Unnamed: 0,dna_chip_id,breed,sex
0,0,202290551164R09C01,Д,Хр
1,1,202341831114R02C01,Д,Хр
2,2,202341831114R03C01,Д,Хр
3,3,202341831114R04C01,Д,Хр
4,4,202290551140R01C01,Д,Хр


,Unnamed: 0,SNP Name,SNP Index,SNP Aux,Sample ID,SNP,Allele1 - Top,Allele2 - Top,Allele1 - Forward,Allele2 - Forward,Allele1 - AB,Allele2 - AB,Chr,Position,GC Score,GT Score,Theta,R,B Allele Freq,Log R Ratio
0,0,1_10573221,1,0,202341831114R01C01,[T/C],-,-,-,-,-,-,1,10573221,0.0000,0.0000,0.942,0.413,1.0000,0.4040
1,1,1_10673082,2,0,202341831114R01C01,[T/C],A,A,T,T,A,A,1,10673082,0.8272,0.8076,0.039,0.968,0.0000,0.3017
2,2,1_10723065,3,0,202341831114R01C01,[A/G],A,A,T,T,A,A,1,10723065,0.8316,0.8107,0.011,1.577,0.0000,0.0388
3,3,1_11337555,4,0,202341831114R01C01,[A/G],A,A,T,T,A,A,1,11337555,0.3781,0.7925,0.045,1.104,0.0000,0.2761
4,4,1_11407894,5,0,202341831114R01C01,[A/G],G,G,G,G,B,B,1,11407894,0.9038,0.8670,0.983,1.122,0.9994,0.0022


Column `Sample ID` contains the same values as `dna_chip_id` => we will use them to connect two tables

In [3]:
all(df_genstudio['Sample ID'].isin(df_metadata['dna_chip_id']))

True

### Create two databases

In [4]:
df_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 841 entries, 0 to 840
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   841 non-null    int64 
 1   dna_chip_id  841 non-null    object
 2   breed        841 non-null    object
 3   sex          841 non-null    object
dtypes: int64(1), object(3)
memory usage: 26.4+ KB


In [5]:
df_genstudio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 20 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   SNP Name           object 
 2   SNP Index          int64  
 3   SNP Aux            int64  
 4   Sample ID          object 
 5   SNP                object 
 6   Allele1 - Top      object 
 7   Allele2 - Top      object 
 8   Allele1 - Forward  object 
 9   Allele2 - Forward  object 
 10  Allele1 - AB       object 
 11  Allele2 - AB       object 
 12  Chr                object 
 13  Position           object 
 14  GC Score           float64
 15  GT Score           float64
 16  Theta              float64
 17  R                  float64
 18  B Allele Freq      float64
 19  Log R Ratio        float64
dtypes: float64(6), int64(3), object(11)
memory usage: 305.2+ MB


In [6]:
connection = sqlite3.connect('../data/my_database.db')
create_metadata = '''CREATE TABLE metadata(
                                Id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
                                dna_chip_id TEXT,
                                breed TEXT,
                                sex TEXT)
                  '''

create_genstudio = '''CREATE TABLE genstudio(
                                        Id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
                                        SNP_name TEXT,
                                        SNP_index INTEGER,
                                        SNP_Aux INTEGER,
                                        dna_chip_id TEXT,
                                        SNP TEXT,
                                        Allele1_Top TEXT,
                                        Allele2_Top TEXT,
                                        Allele1_Forward TEXT, 
                                        Allele2_Forward TEXT,
                                        Allele1_AB TEXT,
                                        Allele2_AB TEXT,
                                        Chr TEXT,
                                        Position TEXT,
                                        GC_Score NUMERIC,
                                        GT_Score NUMERIC,
                                        Theta NUMERIC,
                                        R NUMERIC,
                                        B_Allele_Freq NUMERIC,
                                        Log_R_Ration NUMERIC,
                                        FOREIGN KEY (dna_chip_id) REFERENCES metadata(dna_chip_id))
                     '''
connection.execute(create_metadata)
connection.execute(create_genstudio)

### Fill tables with values

In [7]:
insertion_metadata = '''INSERT INTO metadata(Id,
                                              dna_chip_id,
                                              breed,
                                              sex)
                                        VALUES(?,?,?,?)'''

insertion_genstudio = '''INSERT INTO genstudio(Id,
                                                SNP_name,
                                                SNP_index,
                                                SNP_Aux,
                                                dna_chip_id,
                                                SNP,
                                                Allele1_Top,
                                                Allele2_Top,
                                                Allele1_Forward, 
                                                Allele2_Forward,
                                                Allele1_AB,
                                                Allele2_AB,
                                                Chr,
                                                Position,
                                                GC_Score,
                                                GT_Score,
                                                Theta,
                                                R,
                                                B_Allele_Freq,
                                                Log_R_Ration)
                                        VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'''


connection.executemany(insertion_metadata, df_metadata.values)
connection.executemany(insertion_genstudio, df_genstudio.values)
connection.commit()
connection.close()

### Test some commands

In [10]:
connection = sqlite3.connect('../data/my_database.db')

In [11]:
query = '''SELECT Id, dna_chip_id, SNP_name, SNP FROM genstudio
            WHERE genstudio.GC_score > 0.5'''
rows = connection.execute(query).fetchmany(15)
for row in rows:
    print(row)

(1, '202341831114R01C01', '1_10673082', '[T/C]')
(2, '202341831114R01C01', '1_10723065', '[A/G]')
(4, '202341831114R01C01', '1_11407894', '[A/G]')
(5, '202341831114R01C01', '1_11426075', '[T/C]')
(8, '202341831114R01C01', '1_13996200', '[T/C]')
(9, '202341831114R01C01', '1_142535524', '[A/G]')
(10, '202341831114R01C01', '1_14638936', '[T/C]')
(11, '202341831114R01C01', '1_161891709', '[A/G]')
(12, '202341831114R01C01', '1_17346505', '[A/G]')
(13, '202341831114R01C01', '1_17537210', '[T/C]')
(15, '202341831114R01C01', '1_242598', '[A/G]')
(16, '202341831114R01C01', '1_2463520', '[A/G]')
(18, '202341831114R01C01', '1_286337402', '[A/G]')
(19, '202341831114R01C01', '1_294072400', '[T/C]')
(22, '202341831114R01C01', '1_303127440', '[A/G]')


In [12]:
query = '''SELECT metadata.Id, metadata.dna_chip_id, 
                  metadata.breed, metadata.sex, 
                  genstudio.SNP_name, genstudio.SNP, 
                  genstudio.Chr
          FROM metadata, genstudio
          WHERE metadata.dna_chip_id = genstudio.dna_chip_id    
        '''

rows = connection.execute(query).fetchmany(20)
for row in rows:
    print(row)

(1, '202341831114R02C01', 'Д', 'Хр', '10_11524303', '[A/G]', '10')
(1, '202341831114R02C01', 'Д', 'Хр', '10_15026457', '[T/C]', '10')
(1, '202341831114R02C01', 'Д', 'Хр', '10_15270724', '[A/G]', '10')
(1, '202341831114R02C01', 'Д', 'Хр', '10_17565103', '[A/G]', '10')
(1, '202341831114R02C01', 'Д', 'Хр', '10_17770477', '[A/G]', '10')
(1, '202341831114R02C01', 'Д', 'Хр', '10_18682426', '[T/C]', '10')
(1, '202341831114R02C01', 'Д', 'Хр', '10_19324750', '[T/G]', '10')
(1, '202341831114R02C01', 'Д', 'Хр', '10_20043953', '[T/C]', '10')
(1, '202341831114R02C01', 'Д', 'Хр', '10_20166080', '[A/C]', '10')
(1, '202341831114R02C01', 'Д', 'Хр', '10_2598359', '[A/G]', '10')
(1, '202341831114R02C01', 'Д', 'Хр', '10_27014378', '[T/C]', '10')
(1, '202341831114R02C01', 'Д', 'Хр', '10_27620869', '[A/G]', '10')
(1, '202341831114R02C01', 'Д', 'Хр', '10_28797811', '[T/C]', '10')
(1, '202341831114R02C01', 'Д', 'Хр', '10_28894311', '[T/C]', '10')
(1, '202341831114R02C01', 'Д', 'Хр', '10_29099121', '[T/C]', '1